In [1]:
%matplotlib inline

In [2]:
import fipy as fp

In [3]:
from fipy import numerix as nmx

In [4]:
import os

In [5]:
direc = "1d-exp-restart"

In [6]:
os.mkdir(direc)

OSError: [Errno 17] File exists: '1d-exp-restart'

In [7]:
os.chdir(os.path.join(os.getcwd(), direc))

In [8]:
mesh = fp.dump.read("mesh.dmp")

In [9]:
C = fp.CellVariable(mesh=mesh, name="C", hasOld=True)

In [10]:
Calpha = 0.05
Cbeta = 0.95
Cm = (Calpha + Cbeta) / 2.
A = 2.0
B = A / (Calpha - Cm)**2
D = Dalpha = Dbeta = 2 / (Cbeta - Calpha)
kappa = 2.0

In [11]:
f0 = -(A/2)*(C - Cm)**2 + (B/4)*(C - Cm)**4 + (Calpha/4)*(C - Calpha)**4 + (Cbeta/4)*(C - Cbeta)**4

In [12]:
f = f0 + (kappa/2.)*(C.grad.mag)**2
f.name = "f"

In [13]:
Cf = C.arithmeticFaceValue
eq = (fp.TransientTerm(coeff=1.)
      == fp.DiffusionTerm(coeff=D*(-A + 3*B*(Cf - Cm)**2 + 3*Calpha*(Cf - Calpha)**2 + 3*Cbeta*(Cf - Cbeta)**2))
      - fp.DiffusionTerm(coeff=(D,kappa)))

In [14]:
epsilon = 0.01
theta = nmx.arctan2(mesh.z, nmx.sqrt(mesh.x**2 + mesh.y**2))
phi = nmx.arctan2(mesh.y, mesh.x)
C.value = 0.45 + epsilon * nmx.cos(nmx.sqrt(233.) * theta) * nmx.sin(nmx.sqrt(239.) * phi)

In [ ]:
fp.Viewer(vars=theta).plot()

In [ ]:
fp.Viewer(vars=phi).plot()

In [ ]:
viewer = fp.Viewer(vars=(C))

In [ ]:
viewer.plot()

In [15]:
x, y, z, value = nmx.loadtxt(fname="603.gz", skiprows=2, unpack=True)
C.value = value

In [16]:
with open("stats.txt", 'w') as stats:
    stats.write("\t".join(["step", "t", "dt", "Cmin", "Cmax", "f"]) + "\n")

In [17]:
dt = 2.77319476396
elapsed = 278.037953429
saved = elapsed
duration = 10000.
step = 603

In [18]:
while elapsed < duration:
    C.updateOld()
    dt = 0.0651 * elapsed**(2./3)
    elapsed += dt
    eq.solve(C, dt=dt)
    step += 1
    if elapsed - saved > 10:
        fp.TSVViewer(vars=C).plot(filename="{step}.gz".format(step=step))
        # viewer.plot()
        saved = elapsed
    with open("stats.txt", 'a') as stats:
        stats.write("\t".join([str(it) for it in [step, elapsed, dt, min(C), max(C), 
                                                  f.cellVolumeAverage * mesh.cellVolumes.sum()]]) + "\n")

/Users/guyer/anaconda/envs/fipy_petsc/lib/python2.7/site-packages/ipykernel/__main__.py:5: MaximumIterationWarning: Iterations: 1001. Relative error: 1.24764e-06
/Users/guyer/anaconda/envs/fipy_petsc/lib/python2.7/site-packages/ipykernel/__main__.py:5: MaximumIterationWarning: Iterations: 1001. Relative error: 0.055683
/Users/guyer/anaconda/envs/fipy_petsc/lib/python2.7/site-packages/ipykernel/__main__.py:5: MaximumIterationWarning: Iterations: 1001. Relative error: 8.73272e-05
/Users/guyer/anaconda/envs/fipy_petsc/lib/python2.7/site-packages/ipykernel/__main__.py:5: MaximumIterationWarning: Iterations: 1001. Relative error: 1.21031e-07
/Users/guyer/anaconda/envs/fipy_petsc/lib/python2.7/site-packages/ipykernel/__main__.py:5: MaximumIterationWarning: Iterations: 1001. Relative error: 0.0246807
/Users/guyer/anaconda/envs/fipy_petsc/lib/python2.7/site-packages/ipykernel/__main__.py:5: MaximumIterationWarning: Iterations: 1001. Relative error: 0.278938
/Users/guyer/anaconda/envs/fipy_pets

KeyboardInterrupt: 